# 📘 Optimizing Battery Efficiency Across Devices and Usage Patterns

### Motivation
Portable devices are critical to modern life. Efficient battery usage improves both user experience and hardware longevity. This project aims to analyze how various features like brightness, CPU load, and temperature influence battery performance.

---

## 📂 Dataset Overview
The dataset includes the following columns:
- Date of Measurement
- Device Model
- Operating System Version
- Task Type
- Brightness Level (%)
- Battery Life (hours:minutes)
- Battery Health (%)
- Device Temperature (°C)
- CPU Utilization (%)
- Memory Usage (% RAM)


## 🔧 Data Preprocessing

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
df = pd.read_csv("battery_efficiency_data.csv")

# Convert battery life to minutes
def convert_to_minutes(val):
    if pd.isna(val): return np.nan
    hours, minutes = map(int, val.split(":"))
    return hours * 60 + minutes

df["Battery Life (min)"] = df["Battery Life (hours:minutes)"].apply(convert_to_minutes)
df.drop(columns=["Battery Life (hours:minutes)"], inplace=True)
df.dropna(inplace=True)

# One-hot encode categorical columns
df_encoded = pd.get_dummies(df, columns=["Device Model", "Operating System Version", "Task Type"])
cleaned_df = df_encoded.copy()
df.head()


## 📊 Exploratory Data Analysis (EDA)

In [ ]:

plt.figure(figsize=(12, 10))
sns.heatmap(cleaned_df.corr(numeric_only=True), cmap="coolwarm", annot=False)
plt.title("Correlation Heatmap of Battery Efficiency Features")
plt.tight_layout()
plt.show()


## 📐 Hypothesis Testing
Null Hypothesis: Task Type does not significantly affect battery life.

In [ ]:

from scipy.stats import f_oneway

df_original = pd.read_csv("battery_efficiency_data.csv")
df_original["Battery Life (min)"] = df_original["Battery Life (hours:minutes)"].apply(convert_to_minutes)
df_original.dropna(subset=["Task Type", "Battery Life (min)"], inplace=True)
task_groups = df_original.groupby("Task Type")["Battery Life (min)"].apply(list)
f_stat, p_val = f_oneway(*task_groups)
print(f"F-statistic: {f_stat:.3f}")
print(f"p-value: {p_val:.3f}")


## 🤖 Linear Regression Modeling

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X = cleaned_df.drop(columns=["Date of Measurement", "Battery Life (min)"])
y = cleaned_df["Battery Life (min)"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)

print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))


## 🔍 Classification Modeling (Random Forest)

In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

def classify_life(mins):
    if mins < 240: return "Short"
    elif mins <= 360: return "Medium"
    else: return "Long"

df_class = cleaned_df.copy()
df_class["Battery Life Class"] = df_class["Battery Life (min)"].apply(classify_life)

X_cls = df_class.drop(columns=["Date of Measurement", "Battery Life (min)", "Battery Life Class"])
y_cls = df_class["Battery Life Class"]
le = LabelEncoder()
y_encoded = le.fit_transform(y_cls)

X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X_cls, y_encoded, test_size=0.2, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_cls, y_train_cls)
y_pred_cls = clf.predict(X_test_cls)

print(classification_report(y_test_cls, y_pred_cls, target_names=le.classes_))


## ✅ Conclusion & Future Work
- Brightness, CPU usage, and memory are moderately correlated with lower battery life.
- Linear regression showed poor fit (low R²).
- Classification accuracy was 40%, with better performance for 'Long' battery life.
### Future Improvements:
- Increase dataset size
- Improve class balance for ML tasks
- Try time-based models or deep learning for temporal features